# Read directly from IIASA data resources

The IIASA *Energy, Climate, and Environment* Program hosts a suite of **Scenario Explorer** instances and related infrastructure to support analysis of integrated-assessment pathways in IPCC reports and model comparison projects. 
High-profile use cases include the [IAMC 1.5°C Scenario Explorer hosted by IIASA](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer) supporting the *IPCC Special Report on Global Warming of 1.5°C* (SR15) and the Horizon 2020 project [CD-LINKS](https://data.ene.iiasa.ac.at/cd-links).

IIASA's [modeling platform infrastructure](http://software.ene.iiasa.ac.at/ixmp-server) and the Scenario Explorer UI is not only a great resource on its own, but it also allows the underlying datasets to be directly queried.
**pyam** takes advantage of this ability to allow you to easily pull data and work with it in your Python data processing and analysis workflow.

In [ ]:
import pyam

## Connecting to a data resource (aka the database API of a Scenario Explorer instance)

Accessing a data resource is done via a **Connection** object.
By default, your can connect to all public Scenario Explorer instances. 

In [ ]:
conn = pyam.iiasa.Connection()
conn.valid_connections

If you have credentials to connect to a non-public or restricted Scenario Explorer instance,
you can store this information by running the following command in a separate Python console:

```
import pyam
pyam.iiasa.set_config(<username>, <password>)
```
When initializing a new **Connection** instance, **pyam** will automatically search for the configuration in a known location.

In this example, we will be retrieving data from the *IAMC 1.5°C Scenario Explorer hosted by IIASA*
([link](https://data.ene.iiasa.ac.at/iamc-1.5c-explorer)),
which provides the quantiative scenario ensemble underpinning
the *IPCC Special Report on Global Warming of 1.5C* (SR15).

This can be done either via the constructor:

```
pyam.iiasa.Connection('iamc15')
```

or, if you want to query multiple databases, via the explicit `connect()` method:

```
conn = pyam.iiasa.Connection()
conn.connect('iamc15')
```

We also provide some convenience functions to shorten the amount of code you have to write. Under the hood, `read_iiasa()` is just opening a connection to a database API and sends a query to the resource.

In this tutorial, we will query specific subsets of data in a manner similar to `pyam.IamDataFrame.filter()`.

In [ ]:
df = pyam.read_iiasa(
    'iamc15',
    model='MESSAGEix*', 
    variable=['Emissions|CO2', 'Primary Energy|Coal'], 
    region='World',
    meta=['category']
)

Here we pulled out all times series data for model(s) that start with 'MESSAGEix' that are in the 'World' region and associated with the two named variables. We also added the meta column "category", which tells us the climate impact categorisation of each scenario as assessed in the IPCC SR15.

Let's plot CO2 emissions.

In [ ]:
ax = df.filter(variable='Emissions|CO2').plot(
    color='category', 
    legend=dict(loc='center left', bbox_to_anchor=(1.0, 0.5))
)

And now continue doing all of your analysis!

In [ ]:
ax = df.plot.scatter(
    x='Primary Energy|Coal', 
    y='Emissions|CO2', 
    color='category',
    legend=dict(loc='center left', bbox_to_anchor=(1.0, 0.5))
)

## Exploring the data resource

If you're interested in what data is available in the data source, you can use **pyam.iiasa.Connection** to do so.

In [ ]:
conn = pyam.iiasa.Connection('iamc15')

The **Connection** object has a number of useful functions for listing what's available in the data resource.
These functions follow the conventions of the **IamDataFrame** class (where possible).

A few of them are shown below.

In [ ]:
conn.models().head()

In [ ]:
conn.scenarios().head()

In [ ]:
conn.variables().head()

In [ ]:
conn.regions().head()

A number of different categorization and quantitative indicators are available for model/scenario combinations.  
These are usually called 'meta' indicators in **pyam**.

We queried the meta-indicator "category" in the above example, but there are many more.
You can get a list with the following command:

In [ ]:
conn.meta_columns.head()

You can directly query the **Connection**, which will return a **pyam.IamDataFrame**...

In [ ]:
df = conn.query(
    model='MESSAGEix*', 
    variable=['Emissions|CO2', 'Primary Energy|Coal'], 
    region='World'
)

...so that you can directly continue with your analysis and visualization workflow using **pyam**!

In [ ]:
ax = df.filter(variable='Primary Energy|Coal').plot(
    color='scenario', 
    legend=dict(loc='center left', bbox_to_anchor=(1.0, 0.5))
)